### Tables Of Contents <a class="anchor" id="table_of_contents"></a>

* [Libraries Used](#chapter1)
* [Access to the DB](#chapter2)
* [Table Queries](#chapter3)
* [Q1 - What region had the most units sold for pencil?](#chapter4)
* [Q2 - For each sale above the minimum price the rep gets 10% commission of the total price as a bonus. How many transactions weren’t calculated for commissions?](#chapter5)
* [Q3 - For the item Binder what was the share of each sales rep in the data?](#chapter6)
* [Q4 - For Jardine what was the highest sell, total sells and amount of transactions for each of the items?](#chapter7)
* [Q5 - Calculate per rep the amount of transactions and the amount of transactions only on the items: Pen, Pencil and Pen Set.](#chapter8)
* [Q6 - Which item is the most profitable to sell? Get the percentage of the diff from unit price to minimum and rank it for the entire data?](#chapter9)
* [Q7 - Download the 2 original tables to the same excel file in 2 different sheets and do:](#chapter10)
 - Pivot table that demonstrate amount of units sold per year for each rep
 - Pivot table that demonstrate the maximum transaction per region for each item
 - Use the VLOOKUP function to add a column of minimum price to the rep sales table.



### Used Libraries<a class="anchor" id="chapter1"></a>

In [ ]:
#!pip install psycopg2
#! pip install xlsxwriter

In [2]:
# Import libraries
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

# Configuration 
import warnings
warnings.simplefilter(action='ignore')

### Access to the DB <a class="anchor" id="chapter2"></a>

In [3]:
# Establish connection to training database
db_config = {
                      'user': 'practicum_student',                          # username             
                      'pwd': 's65BlTKV3faNIGhmvJVzOqhs',                    # password             
                      'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',  # host URL      
                      'port': 6432,                                         # connection port             
                      'db': 'data-analyst-sales-data-db'}                   # the name of the database


connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

#  Table Queries <a class="anchor" id="chapter3"></a>

Let's create a function that takes a query and returns the dataframe for general use.

In [4]:
def queryResult(q):
    return pd.io.sql.read_sql(q, con = engine)

* [Top](#table_of_contents)

### Taking a look at the data

Taking a look at all the tables in the data will give us a first impression of what the data is about. Let's have a glance at the first 10 rows of each table.

In [5]:
tables_list  = engine.table_names()

In [6]:
for t in tables_list:
    print('Table {} head'.format(t))
    display(pd.io.sql.read_sql('''SELECT * FROM {} LIMIT 10'''.format(t), con = engine))
    print()

Table item head


,Item,MinPrice
0,Binder,1.99
1,Desk,125.00
2,Pen,1.75
3,Pen Set,4.99
4,Pencil,1.29
5,Marker Set,5.99
6,Binder,1.99
7,Desk,125.00
8,Pen,1.75
9,Pen Set,4.99



Table rep_sales head


,OrderDate,Region,Rep,Item,Units,Unit_Cost,Total
0,2019-01-23,Central,Kivell,Binder,50,19.99,999.50
1,2019-04-01,East,Jones,Binder,60,4.99,299.40
2,2019-06-08,East,Jones,Binder,60,8.99,539.40
3,2019-07-12,East,Howard,Binder,29,1.99,57.71
4,2019-07-29,East,Parent,Binder,52,17.99,935.48
5,2019-10-05,Central,Morgan,Binder,28,8.99,251.72
6,2020-01-15,Central,Gill,Binder,46,8.99,413.54
7,2020-02-01,Central,Smith,Binder,68,14.00,952.00
8,2020-02-18,East,Jones,Binder,4,4.99,19.96
9,2020-03-07,West,Sorvino,Binder,7,19.99,139.93


There apears to be duplicated rows in the item table and we need to keep that in mind working with data from it!


* [Top](#table_of_contents)

## Q1 <a class="anchor" id="chapter4"></a>

1. What region had the most units sold for pencil?

In [7]:
query ='''
SELECT DISTINCT "Region",
        SUM("Units") OVER(PARTITION BY "Region") AS n_sales
FROM rep_sales
WHERE "Item" = 'Pencil'
ORDER BY 2 DESC
LIMIT 1

'''
queryResult(query)

,Region,n_sales
0,Central,498


* [Top](#table_of_contents)

## Q2 <a class="anchor" id="chapter5"></a>
2. For each sale above the minimum price the rep gets 10% commission of the total price as a bonus. How many transactions weren’t calculated for commissions?

The transactions which would not be calculated for comission are the transactions that according to the description "sales BELOW the minimum price" - this is the case when the rep DOESN'T get the 10% comission.
For this query we must first exclude the duplicated rows from the item table.

In [9]:
query = '''
WITH unique_items AS (SELECT DISTINCT "Item", "MinPrice"
                      FROM item)


SELECT COUNT(r."Item") AS no_comission_transaction
FROM rep_sales r 
INNER JOIN unique_items u
ON r."Item" = u."Item"
WHERE "Unit_Cost" <= "MinPrice"

'''
queryResult(query)

,no_comission_transaction
0,15


* [Top](#table_of_contents)

## Q3 <a class="anchor" id="chapter6"></a>
For the item Binder what was the share of each sales rep in the data?

In [10]:
query ='''
SELECT DISTINCT "Rep", 
      ROUND(100 * SUM("Total") OVER(PARTITION BY "Rep") / 
      SUM("Total") OVER(PARTITION BY "Item"), 2) AS share_of_rep_per_item
FROM rep_sales
WHERE "Item" = 'Binder'
ORDER BY 2 DESC;
'''
table_of_share = queryResult(query)
table_of_share

,Rep,share_of_rep_per_item
0,Jones,16.85
1,Gill,13.77
2,Jardine,12.81
3,Kivell,12.15
4,Smith,11.57
5,Parent,11.37
6,Thompson,10.11
7,Joe,4.23
8,Morgan,3.06
9,Andrews,1.70


Representative Jones is the sales leader of Binder item.

* [Top](#table_of_contents)

## Q4 <a class="anchor" id="chapter7"></a>

For Jardine what was the highest sell, total sells and amount of transactions for each of the items?

In [11]:
query ='''
SELECT DISTINCT "Item",
MAX("Total") OVER(PARTITION BY "Item") AS max_sells,
SUM("Total") OVER(PARTITION BY "Item") AS tot_sells,
COUNT("Total") OVER(PARTITION BY "Item") AS n_transactions
FROM rep_sales
WHERE "Rep" = 'Jardine'

'''
queryResult(query)

,Item,max_sells,tot_sells,n_transactions
0,Marker Set,983.18,983.18,1
1,Pen Set,249.50,249.50,1
2,Pencil,449.10,628.74,2
3,Pen,91.29,91.29,1
4,Binder,999.20,1054.09,2


That's a good way to see each representative's sales performance in a more detailed way.

* [Top](#table_of_contents)

## Q5 <a class="anchor" id="chapter8"></a>


5. Calculate per rep the amount of transactions and the amount of transactions only on the items: Pen, Pencil and Pen Set.


In [12]:
query ='''
SELECT "Rep",
        COUNT("OrderDate") AS n_transactions,
        COUNT(CASE WHEN "Item" LIKE '%%Pen%%' THEN 1 ELSE NULL END) AS n_penlike_items
FROM rep_sales
GROUP BY 1
'''
queryResult(query)

,Rep,n_transactions,n_penlike_items
0,Morgan,5,3
1,Gill,7,4
2,Jones,10,6
3,Sorvino,6,2
4,Joe,4,2
5,Jardine,7,4
6,Thompson,6,3
7,Kivell,5,2
8,Andrews,7,4
9,Smith,4,1


If we needed we could also calculate the share of penlike items sold by each rep to get even a more accurate picture.

* [Top](#table_of_contents)

## Q6 <a class="anchor" id="chapter9"></a>

6. Which item is the most profitable to sell? Get the percentage of the diff from unit price to minimum and rank it for the entire data.

In [13]:
query ='''
WITH sub AS (SELECT  "Item", 
        AVG("Unit_Cost") AS avg_unit_cost
        FROM rep_sales
        GROUP BY 1), 

unique_items AS (SELECT DISTINCT "Item", "MinPrice"
                  FROM item)

SELECT DISTINCT s."Item", 
ROUND(((s.avg_unit_cost / u."MinPrice") - 1) * 100 ,2) AS lift
FROM sub s INNER JOIN unique_items u
ON s."Item" = u."Item"
ORDER BY 2 DESC
LIMIT 1;
'''
queryResult(query)

,Item,lift
0,Binder,468.58


The item Binder gives a 468.5% percent lift (trade margin above average) which makes it the most profitable to sell.

* [Top](#table_of_contents)

## Q7 <a class="anchor" id="chapter10"></a>
7. Download the 2 original tables to the same excel file in 2 different sheets and do:
- Pivot table that demonstrate amount of units sold per year for each rep
- Pivot table that demonstrate the maximum transaction per region for each item
- Use the VLOOKUP function to add a column of minimum price to the rep sales table

* [Top](#table_of_contents)

In [18]:
tables_list

['item', 'rep_sales']

In [19]:
# Store the tables entirely in variables
item = pd.io.sql.read_sql('''SELECT * FROM item ''', con = engine)
rep_sales = pd.io.sql.read_sql('''SELECT * FROM rep_sales ''', con = engine)

In [20]:
# Transform the variables into excel file with a separate sheet for each variable
with pd.ExcelWriter("17-17-2022.xlsx", engine = "xlsxwriter") as writer:
    item.to_excel(writer, sheet_name="item", index=False)  
    rep_sales.to_excel(writer, sheet_name="rep_sales", index=False)  

The files are downloaded from the environment and uploaded to Google sheets. They are availiable on [link](https://docs.google.com/spreadsheets/d/10Jt5K3catL6SpedqpWEEAGEbwMSolpWR/edit?usp=sharing&ouid=102290911037493389308&rtpof=true&sd=true). 

To make the most out the answers we found in this research we visualised some of the insights in Tableau. The dashboard is availiable on [this link](https://public.tableau.com/views/SalesAnalysisProject_16594313729070/MyDashboard?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link).